Building a Chatbot
The chatbot will be able to have conversations and remember previous interactions

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() #Load all the environment variables

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [ ]:
from langchain_groq import ChatGroq #Load model from groq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DE4E1F8BF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DE4E1F96A0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi,I am Amritanshu ,studying in Banglore")])

AIMessage(content="Hi Amritanshu,\n\nIt's great to meet you! Bangalore is a fantastic city. What are you studying there? \n\nDo you have any questions for me, or would you like to chat about something related to Bangalore or your studies? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 22, 'total_tokens': 79, 'completion_time': 0.103636364, 'prompt_time': 0.000231769, 'queue_time': 0.015296608, 'total_time': 0.103868133}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3807b867-f447-4fcf-8d34-d94f046e387d-0', usage_metadata={'input_tokens': 22, 'output_tokens': 57, 'total_tokens': 79})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
       HumanMessage(content="Hi,I am Amritanshu ,studying in Banglore"),
       AIMessage(content="Hi Amritanshu,\n\nIt's great to meet you! Bangalore is a fantastic city. What are you studying there?"),
       HumanMessage(content="Hey what's my name and where do i study?")
    ]
)

AIMessage(content="You told me your name is Amritanshu and you study in Bangalore.  \n\nIs there anything else you'd like to talk about? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 70, 'total_tokens': 104, 'completion_time': 0.061818182, 'prompt_time': 0.002595738, 'queue_time': 0.01086474, 'total_time': 0.06441392}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fee60df5-6323-47af-a55b-50bc4d6229d9-0', usage_metadata={'input_tokens': 70, 'output_tokens': 34, 'total_tokens': 104})

### Message History

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

#for different users to have different message history,isme string pass hoga
def get_session_history(session_id:str)->BaseChatMessageHistory: ##Return type is chathistory
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)#to interact with our llm model based on chat history   

In [9]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi,I am Amritanshu ,studying in Banglore")],
    config=config ##matlab session id chat1 ke liye it will remember the context
)

In [12]:
response.content

"Hi Amritanshu! \n\nThat's great to know.  Bangalore is a fantastic city to study in. What are you studying there? 😊 \n\nI'm happy to chat if you have any questions or just want to talk about Bangalore.  \n\n"

In [13]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config ##matlab session id chat1 ke liye it will remember the context
)

AIMessage(content='Your name is Amritanshu. 😊  \n\nHow can I help you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 165, 'total_tokens': 185, 'completion_time': 0.036363636, 'prompt_time': 0.005335668, 'queue_time': 0.007955099, 'total_time': 0.041699304}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-988c61b4-91c9-407a-b307-649a1d800f3a-0', usage_metadata={'input_tokens': 165, 'output_tokens': 20, 'total_tokens': 185})

In [14]:
##Change the config(changing the session id)
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)
response.content

"As a large language model, I have no memory of past conversations and I don't know your name. If you'd like to tell me your name, I'd be happy to use it!\n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.  \n\nIs there anything I can help you with today?\n"

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)
response.content  #with this config id we are able to switch with the contexts

'Your name is John, as you told me earlier. 😊 \n\nDo you have any other questions for me, John? \n'

Prompt Templates

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")##inside this give infomfrom human side in messages
    ]
)
chain = prompt|model

In [19]:
chain.invoke({"messages":[HumanMessage(content="Hi,my name is Amritanshu")]})

AIMessage(content="Hello Amritanshu, it's nice to meet you! \n\nI'm here to help with any questions you have.  What can I do for you today? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 33, 'total_tokens': 74, 'completion_time': 0.074545455, 'prompt_time': 0.000347809, 'queue_time': 0.014421809, 'total_time': 0.074893264}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d4882ca0-0010-4a6d-a3c0-dace968ba40d-0', usage_metadata={'input_tokens': 33, 'output_tokens': 41, 'total_tokens': 74})

In [20]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [21]:
config = {"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi,my name is Amritanshu")],
    config=config,
)
response.content

"Hi Amritanshu, it's nice to meet you! \n\nI'm glad you're here. What can I do for you today? 😊\n\nAsk me anything, and I'll do my best to help!  \n"

In [22]:
#Add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")##inside this give infomfrom human side in messages
    ]
)
chain = prompt|model

In [23]:
response = chain.invoke({"messages":[HumanMessage(content="Hi My name is Amritanshu")],"language":"Hindi"})
response.content

'Namaste Amritanshu! \n\nमुझे बहुत खुशी है आपसे मिलने में।  \n\nआप क्या जानना चाहते हैं? मैं अपनी पूरी क्षमता से आपकी मदद करने के लिए तैयार हूँ। 😊\n'

Lets now wrap more complicated chain in a message history class.This time because their are multiple keys in the input
, we need to specify the correct key to use to save the chat history.

In [24]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [25]:
config={"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi,I am Amritanshu")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते अमितांशु! मैं आपकी मदद करने के लिए यहाँ हूँ। आप क्या जानना चाहते हैं? 😊\n'

In [ ]:
response = with_message_history.invoke(
    {'messages':[HumanMessage(content="What's my name?")],"language":"Hindi"},
    config=config
)
response.content

'आपका नाम अमितांशु है।  \n'

Manage the Conversation History
trim message:helper to reduce how many messages u are sending to the model

In [32]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",  # Corrected typo here
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you are a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!")
]

trimmer.invoke(messages)


[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [33]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)
response=chain.invoke(
    {"messages":messages+[HumanMessage(content="What ice cream do i like")],
    "language":"English"}
)
response.content

"As a helpful assistant, I don't have access to your personal preferences.\n\nTo figure out your favorite ice cream, think about:\n\n\n* **What flavors do you usually gravitate towards?** Chocolate, vanilla, fruity, something unique?\n* **What toppings do you like?** Sprinkles, nuts, chocolate sauce, whipped cream?\n* **What kind of texture do you prefer?** Creamy, chunky, sorbet?\n\n\n\nLet me know if you want to brainstorm some ice cream flavors!\n"

In [34]:
response=chain.invoke(
    {"messages":messages+[HumanMessage(content="What math problem did i ask")],
    "language":"English"}
)
response.content

'You asked me "whats 2+2"  😊  \n\n\n\nLet me know if you have any other questions!\n'

In [36]:
#Lets wrap this in the message history
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable":{"session_id":"chat5"}}

In [41]:
response=with_message_history.invoke(
    {"messages":messages+[HumanMessage(content="What's my name")],
    "language":"English"},
    config=config
)
response.content

"As an AI, I have no memory of past conversations and don't know your name.  What would you like me to call you? 😊  \n"

Vector stores and retrievers

Documents

In [1]:
from langchain_core.documents import Document

documents=[
    Document(
        page_content="Dogs are great companions,known for their loyalty and friendliness",
        metadata={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="Cats and independent pets that often enjoy their own space",
        metadata={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pet for beginners,requiring relatively simple care.",
        meta_data={"source":"fish-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        meta_data={"source":"mammal-pets-doc"},
    ),
]

In [2]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions,known for their loyalty and friendliness'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats and independent pets that often enjoy their own space'),
 Document(metadata={}, page_content='Goldfish are popular pet for beginners,requiring relatively simple care.'),
 Document(metadata={}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm = ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000014728B16240>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000014729C10980>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
#Vector db me store karne se pehle embeddding karni padhti hai
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\Amritanshu Bhardwaj\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#Vector stores
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents, embedding=embeddings)


In [7]:
vectorstore.similarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats and independent pets that often enjoy their own space'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions,known for their loyalty and friendliness'),
 Document(metadata={}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={}, page_content='Goldfish are popular pet for beginners,requiring relatively simple care.')]

In [9]:
##Async query
await vectorstore.asimilarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats and independent pets that often enjoy their own space'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions,known for their loyalty and friendliness'),
 Document(metadata={}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={}, page_content='Goldfish are popular pet for beginners,requiring relatively simple care.')]

In [10]:
vectorstore.similarity_search_with_score("cat")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats and independent pets that often enjoy their own space'),
  1.0272275),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions,known for their loyalty and friendliness'),
  1.5413693),
 (Document(metadata={}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956905),
 (Document(metadata={}, page_content='Goldfish are popular pet for beginners,requiring relatively simple care.'),
  1.7598277)]

Retrievers

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda #retrievers are runnables

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"]) #vector store db ko retrievers me convert kar diya

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats and independent pets that often enjoy their own space')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions,known for their loyalty and friendliness')]]

In [ ]:
retriever = vectorstore.as_retriever(  # document->vectorstore->retriever
    search_type="similarity",
    search_kwargs={"k":1}
)
retriever.batch(["cat","dog"])  #other rway to convert to retriver(best way)

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats and independent pets that often enjoy their own space')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions,known for their loyalty and friendliness')]]

In [ ]:
#this is rag (retrieval augumented generation)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message="""
Answer this question using the provided context only.

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human",message)])

rag_chain = {"context":retriever,"question":RunnablePassthrough()}|prompt|llm
response=rag_chain.invoke("tell me about dogs")
print(response.content)

According to the provided context, dogs are great companions, known for their loyalty and friendliness.
